In [1]:
from neo4j import GraphDatabase
import csv
import ast
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor
import pandas as pd

In [2]:
uri = "neo4j+s://46397e8a.databases.neo4j.io:7687"  # Change this to your Neo4j instance URI
username = "neo4j"             # Replace with your username
password = "apKGp4_QOz5YJtsku_MyYDFBdF7LAEsyyVrKhSaClZk"          # Replace with your password
driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
## Importing entities onto neo4j
def create_graph(entities_file, cooccurrences_file):
    # Create nodes and relationships in Neo4j
    with driver.session() as session:
        print("Conenction Established!")
        # Load entities
        with open(entities_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                print(row)
                # Create entity nodes
                session.run("""
                    MERGE (e:Entity {name: $name, type: $type})
                """, name=row['entity'], type=row['type'])
                
        # Load co-occurrences
        with open(cooccurrences_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                # Create relationships between entities
                session.run("""
                    MATCH (e1:Entity {name: $entity1}), (e2:Entity {name: $entity2})
                    MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
                    ON CREATE SET r.weight = $weight
                    ON MATCH SET r.weight = r.weight + $weight
                """, entity1=row['entity1'], entity2=row['entity2'], weight=int(row['weight']))
def create_graph_parallel(entities_file, cooccurrences_file, batch_size=1000):
    with driver.session() as session:
        print("Conenction Established!")
        # Load entities
        with open(entities_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                print(row)
                # Create entity nodes
                session.run("""
                    MERGE (e:Entity {name: $name, type: $type})
                """, name=row['entity'], type=row['type'])
        
        with open(cooccurrences_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            batch = []
            for row in csv_reader:
                # Append each row to the batch
                batch.append({
                    'entity1': row['entity1'],
                    'entity2': row['entity2'],
                    'weight': int(row['weight'])
                })

                # Process batch when size reaches batch_size
                if len(batch) == batch_size:
                    _process_batch(session, batch)
                    batch = []  # Reset batch

            # Process remaining rows
            if batch:
                _process_batch(session, batch)
def _process_batch(session, batch):
    # Helper function to process a batch of rows
    session.write_transaction(_write_relationships, batch)

def _write_relationships(tx, batch):
    # Use an UNWIND query to handle multiple rows in a single transaction
    query = """
        UNWIND $rows AS row
        MATCH (e1:Entity {name: row.entity1}), (e2:Entity {name: row.entity2})
        MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
        ON CREATE SET r.weight = row.weight
        ON MATCH SET r.weight = r.weight + row.weight
    """
    tx.run(query, rows=batch)
# Specify the file paths
entities_file = "/home/vidur/mediagraph/Python/cleaned_entities.csv"
cooccurrences_file = "/home/vidur/mediagraph/Python/cleaned_cooccurrences_dates2.csv"

# Create the graph
create_graph_parallel(entities_file, cooccurrences_file)

# Close the driver after operations
driver.close()


Conenction Established!
{'entity': 'Kanadia village', 'type': 'GPE'}
{'entity': 'M Manohar Reddy', 'type': 'PERSON'}
{'entity': 'Puvayan', 'type': 'GPE'}
{'entity': 'Tobacco', 'type': 'ORG'}
{'entity': 'Moti', 'type': 'PERSON'}
{'entity': 'Sawaikar', 'type': 'PERSON'}
{'entity': 'Lok Bandhu', 'type': 'PERSON'}
{'entity': 'Niti Aayog', 'type': 'ORG'}
{'entity': 'Jalmuru', 'type': 'GPE'}
{'entity': 'Harpanahalli', 'type': 'GPE'}
{'entity': 'Behvalpur', 'type': 'GPE'}
{'entity': 'Supreme Court', 'type': 'ORG'}
{'entity': 'Rashidpur village', 'type': 'GPE'}
{'entity': 'Bajawala', 'type': 'GPE'}
{'entity': 'Shahul Hameed', 'type': 'DIR'}
{'entity': 'the Apex Bank', 'type': 'ORG'}
{'entity': 'Bharatiya Kisan Sangam', 'type': 'ORG'}
{'entity': 'Sunil Kumar Singh', 'type': 'POL'}
{'entity': 'Ochlandra', 'type': 'PERSON'}
{'entity': 'Rangampet', 'type': 'GPE'}
{'entity': 'Rajesh Khirsagar', 'type': 'PERSON'}
{'entity': 'Shishupal', 'type': 'POL'}
{'entity': 'Abhel Charolia', 'type': 'PERSON'}
{

/tmp/ipykernel_954554/1505534082.py:61: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(_write_relationships, batch)
